In [2]:
# from msilib import sequence
import os
import gensim
from collections import Counter
import json
import torch

sequence_length = 20
train_path = "./Dataset/yelp-subset.train.csv"
test_path = "./Dataset/yelp-subset.test.csv"
#setup Glove word embeddings
with open('glove.6B/glove.6B.50d.txt', 'r') as f:
    lines = f.readlines()

glove = dict()

for line in lines:
    items = line.split()
    word = items[0]
    vector = list(map(float, items[1:]))
    glove[word] = vector

def build_vocab(path, min_word_count = 20):
	counter = Counter()

	with open(path, 'r') as f:
		lines = f.readlines()
	processedLines = [gensim.utils.simple_preprocess(sentence,min_len=1) for sentence in lines]
	for line in processedLines:
		counter.update(line)
	#initialise a dictionary or look up table
	word2id = {}
	word2id['<pad>'] = 0
	word2id['<unk>'] = 1
	# include only those in dictionary which have occered more than min word count in the entire data.
	words = [word for word, count in counter.items() if count>min_word_count]

	for i, word in enumerate(words):
		word2id[word] = i+2
	return word2id


word2id = build_vocab(train_path)
print("Dictionary Formed and saved. The length of dictionary is-: ", len(word2id))

Dictionary Formed and saved. The length of dictionary is-:  9773


In [13]:

from torch.utils.data import Dataset, DataLoader
class Dataset_seq(Dataset):
	def __init__(self, word2id, train_path):
		self.word2id = word2id
		self.train_path = train_path
		# read the data and label 

		with open(train_path, 'r') as f:
			lines = f.readlines()
		processedLines = [gensim.utils.simple_preprocess(sentence,min_len=1) for sentence in lines]
		self.data = processedLines
		self.X = []
		self.Y = []
		for sent in self.data:

			if len(sent) > sequence_length:
				for j in range(len(sent)//sequence_length):
					self.X.append(sent[j*sequence_length:(j+1)*sequence_length])

			elif len(sent) < sequence_length:
				self.X.append(sent + (sequence_length - len(sent)) * ['<pad>'])
		


	def __getitem__(self, index):
		# return the seq and label 
		return seq, 

	def __len__(self):
		return(len(self.data))

q = Dataset_seq(word2id, train_path)


In [16]:
print(q.X[11])

['try', 'something', 'if', 'you', 're', 'in', 'the', 'area', 'but', 'you', 'll', 'want', 'to', 'get', 'there', 'early', 'as', 'like', 'many', 'bakeries']


In [5]:
import torch.nn as nn 
class Bi_RNN(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=11, num_layers=2):
        super(Bi_RNN, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers


        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True, bidirectional=True)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim * 2, output_dim)


    def forward(self, input):
        #Forward pass through initial hidden layer
        linear_input = self.init_linear(input)

        # Forward pass through LSTM layer
        # shape of lstm_out: [batch_size, input_size ,hidden_dim]
        # shape of self.hidden: (a, b), where a and b both
        # have shape (batch_size, num_layers, hidden_dim).
        lstm_out, self.hidden = self.lstm(linear_input)

        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out)
        return F.log_softmax(y_pred, dim=1), self.hidden